In [11]:
import os
import pandas as pd
from QUANTAXIS.QAUtil.QADate import QA_util_datetime_to_strdate as date_to_str
from QUANTAXIS import QA_fetch_index_day_adv
from QUANTAXIS import QA_fetch_stock_day_adv
from QUANTAXIS import QA_fetch_stock_list
import pyfolio as pf
import numpy as np
import jupyter_helper

In [16]:
%%html
<style>
.rendered_html tr,.rendered_html th,.rendered_html td {text-align:left}
</style>

## 年度分析数据列名说明


|列名|说明|
|---|---|
|Annual return|年利润。年度回报是投资在一年内为投资者赚取的金额。|
|Cumulative returns|累积回报。累积回报是投资为投资者赚取的全部金额，无论时间如何。|
|Annual volatility|年化波动率|
|Sharpe ratio|夏普比率|夏普指数假定基金的收益率遵循正态分布，对于非正态分布的情况难以进行有效的衡量，而Stutzer指数考虑收益率分布的偏斜度(Skewness)和峰度(Kurtosis)，偏斜度为正、峰度较小的基金业绩更好。在正态分布的情况下，Stutzer指数和夏普指数的结果是一样的。|
|Calmar ratio|Calmar比率使用略微修改的英镑比率 - 过去36个月的平均年回报率除以过去36个月的最大亏损 - 并按月计算，而不是按年度比率计算。<br/>Calmar比率(Calmar Ratio) 描述的是收益和最大回撤之间的关系。计算方式为年化收益率与历史最大回撤之间的比率。Calmar比率**数值越大，基金的业绩表现越好。反之，基金的业绩表现越差。**|
|Stability|稳定性|
|Max drawdown|指帳戶淨值從最高點的滑落程度，意義在於，從任一時間點進場可能遇到的最糟狀況。<br/>MDD 作為評測策略的標準之一，但不盡然 MDD 越小就越好，投資人在選擇投資策略時，應該將 MDD 視為該策略的風險，就自己能力可以承擔的風險下，選擇最好的投資策略。|
|Omega ratio|该比率是广泛使用的夏普比率的替代方案，并且基于夏普比率丢弃的信息。<br/>通过在累积回报分布中创建分区来计算Omega，以便创建损失区域和相对于该阈值的增益区域。<br/>较大的比率表明资产相对于某些阈值r的损失提供了更多的收益，因此投资者更喜欢这种收益。|
|Sortino ratio|索提诺比率（Sortino Ratio）与夏普比率类似，所不同的是它区分了波动的好坏，因此在计算波动率时它所采用的不是标准差，而是下行标准差。这其中的隐含条件是投资组合的上涨（正回报率）符合投资人的需求，不应计入风险调整。<br/>索提诺比率和夏普比率类似，这一**比率越高，表明基金承担相同单位下行风险能获得更高的超额回报率。**<br/>和夏普比率相比，索提诺比率这一衡量标准更符合那些对资产价值下跌较为敏感的投资者。|
|Skew|偏度是一组数据中对称钟形曲线或正态分布的失真程度。偏度可以是负的，正的，零的或未定义的。<br/>偏度分为两种：<br/>负偏态或左偏态：左侧的尾部更长，分布的主体集中在右侧。<br/>正偏态或右偏态：右侧的尾部更长，分布的主体集中在左侧。<br/>如果分布对称，那么平均值=中位数，偏度为零（此外，如果分布为单峰分布，那么平均值=中位数=众数）。<br/>在金融市场，峰度大于0表现为无趋势，峰度小于0表现为趋势市；偏度大于0可以解释为数据倾向于汇聚成正的趋势，偏度小于0可以解释为数据倾向于汇聚成负的趋势。|
|Kurtosis|峰度可以测量尾部的极端值。具有大峰度的分布显示尾部数据超过正态分布的尾部（例如，与平均值的五个或更多个标准偏差）。具有低峰度的分布表现出尾部数据，其通常不如正态分布的尾部那么极端。<br/>对于投资者而言，回报分布的高峰值意味着投资者将偶尔经历极端回报（正面或负面），比通常的+或 - 三个标准偏差更加极端，这是通过正常回报分布预测的平均值。这种现象称为峰度风险。<br/>Kurtosis的类型<br/>可以通过一组数据显示三种峰度。将峰度的所有测量值与标准正态分布或钟形曲线进行比较。<br/>第一类峰度是中间分布。该分布具有类似于正态分布的峰度统计量，这意味着分布的极值特征类似于正态分布的极值特征。<br/>第二类是leptokurtic分配。任何leptokurtic的分布都显示出比中间分布更大的峰度。这种类型的分布的特征是具有长尾（异常值）的分布。“lepto-”的前缀意味着“瘦”，使得leptokurtic分布的形状更容易记住。leptokurtic分布的“skinniness”是异常值的结果，其延伸直方图的水平轴，使得大部分数据出现在狭窄（“瘦”）的垂直范围内。因此，有些人将leptokurtic分布描述为“集中于均值”，但更相关的问题（特别是对投资者而言）是偶尔存在导致这种“集中”外观的极端异常值。leptokurtic分布的例子是具有小自由度的T分布。<br/>最终的分布类型是platykurtic分布。这些类型的分布具有短尾（缺乏异常值）。“platy-”的前缀意味着“广泛”，它意味着描述一个短而宽阔的峰值，但这是一个历史错误。均匀分布是扁平的并且具有宽峰，但β（.5,1）分布也是扁平的并且具有无限尖峰。这些分布都是platykurtic的原因是它们的极值小于正态分布。**对于投资者而言，platykurtic回报分布是稳定且可预测的，因为很少（如果有）极端（异常）回报。**|
|Tail ratio|尾部风险（Tail Risk）指的是一种资产组合风险，当所建立的投资组合收益偏离均值多于三个标准差的概率大于正态分布所拥有的概率时，尾部风险就产生了。尾部风险产生的概率较小，并且在分布的尾端产生。<br/?通常我们假设市场收益遵循正态分布，并且传统的投资组合管理策略都以此为基础。在这个假设下，投资收益大概率位于偏离均值三个标准差的区间内，无论正负，此概率约为99.97%。也就是说，在正态分布假设下，投资收益偏离均值超过三个标准差的概率仅仅只有0.03%。如下图所示，μ代表均值，左右两根小竖线代表偏离均值三个标准差的位置，那么夹在两根竖线中的这块区域面积为99.97%，而剩余的区域面积即为0.03%。<br/><img src="http://5b0988e595225.cdn.sohucs.com/images/20170823/5cfc9dd91b9347939fa6a10383b2a291.png"><br/>然而尾部风险的提出就是因为实际投资收益并不完全遵循正态分布，而是有偏斜的，也就是说，存在肥尾。而肥尾的存在使得投资收益偏离均值三个标准差的概率是大于0.03%的，这使得投资得到亏损的概率也相应增加。谨慎的资产组合管理者都会尽量避免尾部风险，因为这些风险会带来潜在的巨额损失甚至毁了整个投资组合。下图中的蓝线代表了正态分布，绿线代表了肥尾存在的分布，图中可知，左边虚线外的面积对比中，绿线所包含的要比蓝线的大，因此存在尾部风险。<br/><img src="http://5b0988e595225.cdn.sohucs.com/images/20170823/aeb3f30875914c64800c5dbbc977ccf6.jpeg"/><br/>虽然尾部风险比较少见，但一旦发生就会带来很大的损失。因此，投资者要尽量对冲这些风险。对冲尾部风险意味着保持长期收益，但会牺牲一部分短期收益。举个例子，如果一个投资人追踪大盘指数，那么他就要同时买进一个与大盘指数完全负相关的金融产品来进行对冲。|
|Daily value at risk|日在险价值（Value at Risk，缩写VaR）。VaR定义为在一段时间（例如，1天）内具有特定置信水平（例如，95％）的预测的最坏情况损失。相比于以标准差为代表的传统风险测度，VaR着重于考虑更重要的资产下行风险，特别是极端情况下的损失规模。<br/> VaR(Value at Risk)中文稱為風險值，簡單來說，VaR 代表了在一個特定時間內，正常的市場狀況下，最大的投資損失將有多少？使用上通常會加上時間單位(如：月、10 日、週、年等)及信賴水準(通常是95%或是99%)兩大要素。<br/>舉例來說，常用到的VaR 為每日VaR，其信賴水準為95%。就白話來說，當Daily VaR (95%)=1.5%，代表這檔基金每天所承受的風險，有95%的機率小於淨值的1.5%，也可以換個方式說，在每100 個交易日當中，有95 個交易日基金所承受的風險波動度小於淨值的1.5%。<br/>涉及的時間單位越長，因為市場不確定性越高，所代表的VaR 也越大，實務上適用於流動性比較差的市場，反之亦然，一般則以一日VaR 最為常用；此外，信賴水準越高代表樣本涵蓋率越廣，也意味著管理者或投資者希望得到更為穩健保守的預估結果，如此則VaR 數字也會比較大。<br/>除了使用VaR 來代表基金一日的風險變動值以外，更可以進一步的擴大解釋為操作積極度，因為當VaR 越高的時候，代表可操作標的物的廣度或深度都加大了，當然獲利的機會也變多，因此實務上可以用比較正面的態度去解讀VaR 所隱含的潛在意義。<br/>台灣股市目前Daily VaR 為3%左右，而Daily VaR定義上落在2%以上為積極區，1%以下為保守區，可以幫助投資人爾後有效的解讀並運用這項數據。|
|Alpha|Alpha系数。绝对回报(Absolute Return) 和按照$\beta$系数计算的预期回报之间的差额。是一种收益指标。**越大越好**。<br/>$\alpha>0$，表示一基金或股票的价格可能被低估，建议买入。亦即表示该基金或股票以投资技术获得平均比预期回报大的实际回报。<br/>$\alpha<0$，表示一基金或股票的价格可能被高估，建议卖空。亦即表示该基金或股票以投资技术获得平均比预期回报小的实际回报。<br/>$\alpha=0$，表示一基金或股票的价格准确反映其内在价值，未被高估也未被低估。亦即表示该基金或股票以投资技术获得平均与预期回报相等的实际回报。|
|Beta|Beta系数。用以度量一项资产系统性风险的指标。**越小越好**。<br/>贝塔系数等于1即证券的价格与市场一同变动。（ $\left|\beta \right|=1$）<br/>贝塔系数高于1即证券价格比总体市场更波动。（ $\left|\beta \right|>1$）<br/>贝塔系数低于1即证券价格的波动比市场为低。（ $\left|\beta \right|<1$）<br/>如果 $\beta=0$  表示没有风险， $\beta=0.5$  表示其风险仅为市场的一半， $\beta=1$  表示风险与市场风险相同， $\beta=2$ 表示其风险是市场的2倍。|

## 读取所有股票的年度分析数据

In [6]:
lst=QA_fetch_stock_list()

s,e=jupyter_helper.get_start_end_date()

np.round(jupyter_helper.get_year_stats(date_to_str(s),date_to_str(e),lst=lst.code.values),4)

QA Error QA_fetch_stock_day_adv parameter code=002946 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=002947 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300755 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300758 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300759 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=600928 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601298 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601615 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None


Annual return  Cumulative returns  Annual volatility  \
code   year                                                         
000001 2005  -1.417000e-01             -0.0957             0.3795   
       2006   1.624500e+00              1.3041             0.4283   
       2007   2.659400e+00              2.0092             0.6069   
       2008  -6.885000e-01             -0.6752             0.6624   
       2009   1.728800e+00              1.5098             0.4840   
       2010  -4.151000e-01             -0.3340             0.3690   
       2011  -2.880000e-02             -0.0268             0.2678   
       2012   7.040000e-02              0.0646             0.2344   
       2013   2.546000e-01              0.2367             0.4952   
       2014   6.024000e-01              0.5756             0.3305   
       2015  -9.720000e-02             -0.0939             0.4516   
       2016  -2.270000e-02             -0.0219             0.1845   
       2017   4.945000e-01              0.4732             0.2599   
       2018  -3.149000e-01             -0.3046             0.3373   
000002 2005   2.476000e-01              0.1427             0.3603   
       2006   2.711300e+00              2.3971             0.4585   
       2007   2.031500e+00              1.8129             0.6338   
       2008  -6.577000e-01             -0.6443             0.6633   
       2009   6.572000e-01              0.6210             0.4486   
       2010  -2.269000e-01             -0.2166             0.3612   
       2011  -1.462000e-01             -0.1403             0.2672   
       2012   4.308000e-01              0.4006             0.2882   
       2013  -2.926000e-01             -0.2669             0.3783   
       2014   8.722000e-01              0.8353             0.3581   
       2015   7.637000e-01              0.6937             0.4945   
       2016  -5.370000e-02             -0.0264             0.5369   
       2017   5.897000e-01              0.5493             0.3833   
       2018  -2.484000e-01             -0.2398             0.4234   
000004 2005  -3.749000e-01             -0.2837             0.5036   
       2006   2.749000e-01              0.2232             0.4254   
...                    ...                 ...                ...   
603997 2017  -1.411000e-01             -0.1338             0.3534   
       2018  -4.574000e-01             -0.3084             0.3876   
603998 2005            NaN                 NaN                NaN   
       2006            NaN                 NaN                NaN   
       2007            NaN                 NaN                NaN   
       2008            NaN                 NaN                NaN   
       2009            NaN                 NaN                NaN   
       2010            NaN                 NaN                NaN   
       2011            NaN                 NaN                NaN   
       2012            NaN                 NaN                NaN   
       2013            NaN                 NaN                NaN   
       2014   1.100145e+03              0.6492             1.1370   
       2015   1.147800e+00              0.9491             0.7210   
       2016   1.618000e-01              0.1556             0.5205   
       2017  -3.862000e-01             -0.3754             0.2937   
       2018  -6.854000e-01             -0.5799             0.4706   
603999 2005            NaN                 NaN                NaN   
       2006            NaN                 NaN                NaN   
       2007            NaN                 NaN                NaN   
       2008            NaN                 NaN                NaN   
       2009            NaN                 NaN                NaN   
       2010            NaN                 NaN                NaN   
       2011            NaN                 NaN                NaN   
       2012            NaN                 NaN                NaN   
       2013            NaN                 NaN                NaN   
       2014

limit_output extension: Maximum message size of 10000 exceeded with 24850 characters

## 读取指定股票的年度分析数据

In [9]:
np.round(jupyter_helper.calc_perf_stats_by_year('000002',jupyter_helper.get_benchmark_code(),s,e),4)

Annual return  Cumulative returns  Annual volatility  \
code   year                                                         
000002 2005         0.2476              0.1427             0.3603   
       2006         2.7113              2.3971             0.4585   
       2007         2.0315              1.8129             0.6338   
       2008        -0.6577             -0.6443             0.6633   
       2009         0.6572              0.6210             0.4486   
       2010        -0.2269             -0.2166             0.3612   
       2011        -0.1462             -0.1403             0.2672   
       2012         0.4308              0.4006             0.2882   
       2013        -0.2926             -0.2669             0.3783   
       2014         0.8722              0.8353             0.3581   
       2015         0.7637              0.6937             0.4945   
       2016        -0.0537             -0.0264             0.5369   
       2017         0.5897              0.5493             0.3833   
       2018        -0.2484             -0.2398             0.4234   

             Sharpe ratio  Calmar ratio  Stability  Max drawdown  Omega ratio  \
code   year                                                                     
000002 2005        0.7925        0.9129     0.4324       -0.2712       1.1453   
       2006        3.0922       10.5727     0.7056       -0.2564       1.6796   
       2007        2.0674        6.4712     0.8608       -0.3139       1.4037   
       2008       -1.2814       -0.8960     0.8602       -0.7340       0.8192   
       2009        1.3499        1.8804     0.5690       -0.3495       1.2632   
       2010       -0.5331       -0.6245     0.0398       -0.3634       0.9104   
       2011       -0.4576       -0.6428     0.5247       -0.2274       0.9255   
       2012        1.3870        2.0520     0.2172       -0.2099       1.2641   
       2013       -0.7258       -0.7798     0.7932       -0.3752       0.8835   
       2014        1.9294        4.9076     0.8707       -0.1777       1.4070   
       2015        1.3923        3.7831     0.3651       -0.2019       1.2740   
       2016        0.1584       -0.2006     0.3133       -0.2677       1.0307   
       2017        1.3969        3.3799     0.7899       -0.1745       1.3210   
       2018       -0.4634       -0.5040     0.5942       -0.4928       0.9261   

             Sortino ratio    Skew  Kurtosis  Tail ratio  Daily value at risk  \
code   year                                                                     
000002 2005         1.1895  0.2034    1.3710      1.1578              -0.0443   
       2006         5.7130  0.5748    0.7606      1.4777              -0.0521   
       2007         3.3071  0.1024    0.3696      1.5286              -0.0747   
       2008        -1.7209  0.0922   -0.2783      0.9438              -0.0869   
       2009         2.0885  0.1138    1.4602      1.1798              -0.0541   
       2010        -0.7697  0.4171    2.7703      1.0163              -0.0463   
       2011        -0.6237 -0.2331    1.2239      1.0249              -0.0342   
       2012         2.1645  0.0887    0.9142      1.2571              -0.0347   
       2013        -1.0287  0.0983    2.0619      1.3928              -0.0488   
       2014         3.3989  0.7876    2.8794      1.8151              -0.0424   
       2015         2.2672  0.4725    1.7991      1.3115              -0.0596   
       2016         0.2613  0.9491    2.5110      1.7330              -0.0673   
       2017         2.6051  1.4149    4.8387      1.4010              -0.0462   
       2018        -0.6583  0.1383    1.0164      1.1124              -0.0541   

              Alpha    Beta  
code   year                  
000002 2005  0.3411  1.0379  
       2006  0.3195  1.2445  
       2007  0.1028  1.1418  
       2008  0.2929  1.0937  
       2009 -0.1318  1.0691  
       2010 -0.1368  0.9728  
       2011  0.1565  0.9343  
       2012  0.3084  0.9531  
       2013 -0